# Lojistik Regresyon için Gradyan İnişi


In [1]:
import copy, math
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from CourseFunctions.lab_utils_common import  dlc, plot_data, plt_tumor_data, sigmoid, compute_cost_logistic
from CourseFunctions.plt_quad_logistic import plt_quad_logistic, plt_prob
plt.style.use('./deeplearning.mplstyle')

## Veri Kümesi
Karar sınırı laboratuvarında kullanılan aynı iki özellikli veri kümesiyle başlayalım.


In [2]:
X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_train = np.array([0, 0, 0, 1, 1, 1])

Daha önce olduğu gibi, bu verileri görselleştirmek için bir yardımcı fonksiyon kullanacağız. Etiketi $y=1$ olan veri noktaları kırmızı çarpılar olarak, etiketi $y=0$ olan veri noktaları ise mavi daireler olarak gösterilecektir.


In [3]:
fig,ax = plt.subplots(1,1,figsize=(4,4))
plot_data(X_train, y_train, ax)

ax.axis([0, 4, 0, 3.5])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.show()

<IPython.core.display.Javascript object>

## Lojistik Gradyan İnişi
<img align="right" src="./image/Logistic_gradient_descent.png" style=" width:400px; padding: 10px; " >

Gradyan inişi algoritmasının gradyan hesaplamasını kullandığını hatırlayın:
$$\begin{align*}
&\text{dönüşüm tamamlanana kadar tekrar et:} \; \lbrace \\
&  \; \; \;w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{j := 0..n-1 için} \\
&  \; \; \;  \; \;b = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \\
&\rbrace
\end{align*}$$

Her bir iterasyon, $j$ için tüm $w_j$'yi eşzamanlı olarak günceller ve
$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3}
\end{align*}$$

* $m$ veri kümesindeki eğitim örneklerinin sayısıdır
* $f_{\mathbf{w},b}(x^{(i)})$ modelin tahmini, $y^{(i)}$ ise hedef değeri temsil eder
* Lojistik regresyon modeli için
    $z = \mathbf{w} \cdot \mathbf{x} + b$
    $f_{\mathbf{w},b}(x) = g(z)$
    burada $g(z)$ sigmoid fonksiyonudur:
    $g(z) = \frac{1}{1+e^{-z}}$


### Gradyan İnişi Uygulaması
Gradyan inişi algoritmasının uygulaması iki bileşenden oluşur:
- Yukarıdaki denklem (1)'i uygulayan döngü. Bu, aşağıdaki `gradient_descent` fonksiyonudur ve genellikle opsiyonel ve uygulama laboratuvarlarında size sağlanır.
- Mevcut gradyanın hesaplanması, yukarıdaki denklemler (2,3). Bu, aşağıdaki `compute_gradient_logistic` fonksiyonudur. Bu hafta uygulama laboratuvarınızı yazmanız istenecektir.

#### Gradyanın Hesaplanması, Kod Açıklaması
Yukarıdaki denklem (2) ve (3)'ü tüm $w_j$ ve $b$ için uygular.
Bunu uygulamanın birçok yolu vardır. Aşağıda bir yaklaşım özetlenmiştir:
- `dj_dw` ve `dj_db` değerlerini biriktirecek değişkenleri başlatın
- Her örnek için
    - O örnek için hatayı hesaplayın $g(\mathbf{w} \cdot \mathbf{x}^{(i)} + b) - \mathbf{y}^{(i)}$
    - Bu örnekteki her bir giriş değeri $x_{j}^{(i)}$ için,
        - Hata ile giriş değerini çarpın $x_{j}^{(i)}$ ve sonucu `dj_dw`'in karşılık gelen öğesine ekleyin. (yukarıdaki denklem 2)
    - Hata değerini `dj_db`'ye ekleyin (yukarıdaki denklem 3)

- `dj_db` ve `dj_dw`'yi toplam örnek sayısı (m) ile bölün
- $\mathbf{x}^{(i)}$ numpy'da `X[i,:]` veya `X[i]` olarak ve $x_{j}^{(i)}$ ise `X[i,j]` olarak temsil edilir


In [6]:
def compute_gradient_logistic(X, y, w, b):
    """
    Computes the gradient for logistic regression

    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters
      b (scalar)      : model parameter
    Returns
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w.
      dj_db (scalar)      : The gradient of the cost w.r.t. the parameter b.
    """
    m,n = X.shape
    dj_dw = np.zeros((n,))                           #(n,)
    dj_db = 0.

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i],w) + b)          #(n,)(n,)=scalar
        err_i  = f_wb_i  - y[i]                       #scalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]      #scalar
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                   #(n,)
    dj_db = dj_db/m                                   #scalar

    return dj_db, dj_dw

Gradyan fonksiyonunun uygulanmasını aşağıdaki hücreyi kullanarak kontrol edin.


In [7]:
X_tmp = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_tmp = np.array([0, 0, 0, 1, 1, 1])
w_tmp = np.array([2.,3.])
b_tmp = 1.
dj_db_tmp, dj_dw_tmp = compute_gradient_logistic(X_tmp, y_tmp, w_tmp, b_tmp)
print(f"dj_db: {dj_db_tmp}" )
print(f"dj_dw: {dj_dw_tmp.tolist()}" )

dj_db: 0.49861806546328574
dj_dw: [0.498333393278696, 0.49883942983996693]


#### Gradyan İnişi Kodu
Yukarıdaki denklem (1)'i uygulayan kod aşağıda verilmiştir. Fonksiyonları bulun ve yukarıdaki denklemlerle karşılaştırmak için bir süre vakit ayırın.


In [9]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters):
    """
    Performs batch gradient descent

    Args:
      X (ndarray (m,n)   : Data, m examples with n features
      y (ndarray (m,))   : target values
      w_in (ndarray (n,)): Initial values of model parameters
      b_in (scalar)      : Initial values of model parameter
      alpha (float)      : Learning rate
      num_iters (scalar) : number of iterations to run gradient descent

    Returns:
      w (ndarray (n,))   : Updated values of parameters
      b (scalar)         : Updated value of parameter
    """
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in

    for i in range(num_iters):
        # Calculate the gradient and update the parameters
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b)

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion
            J_history.append( compute_cost_logistic(X, y, w, b) )

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")

    return w, b, J_history         #return final w,b and J history for graphing


Veri setimiz üzerinde gradyan inişi uygulayalım.


In [10]:
w_tmp  = np.zeros_like(X_train[0])
b_tmp  = 0.
alph = 0.1
iters = 10000

w_out, b_out, _ = gradient_descent(X_train, y_train, w_tmp, b_tmp, alph, iters)
print(f"\nupdated parameters: w:{w_out}, b:{b_out}")

Iteration    0: Cost 0.684610468560574   
Iteration 1000: Cost 0.1590977666870457   
Iteration 2000: Cost 0.08460064176930078   
Iteration 3000: Cost 0.05705327279402531   
Iteration 4000: Cost 0.04290759421682   
Iteration 5000: Cost 0.03433847729884557   
Iteration 6000: Cost 0.02860379802212006   
Iteration 7000: Cost 0.02450156960879306   
Iteration 8000: Cost 0.02142370332569295   
Iteration 9000: Cost 0.019030137124109114   

updated parameters: w:[5.28 5.08], b:-14.222409982019837


#### Gradyan inişi sonuçlarını çizelim:


In [11]:
fig,ax = plt.subplots(1,1,figsize=(5,4))
# plot the probability
plt_prob(ax, w_out, b_out)

# Plot the original data
ax.set_ylabel(r'$x_1$')
ax.set_xlabel(r'$x_0$')
ax.axis([0, 4, 0, 3.5])
plot_data(X_train,y_train,ax)

# Plot the decision boundary
x0 = -b_out/w_out[0]
x1 = -b_out/w_out[1]
ax.plot([0,x0],[x1,0], c=dlc["dlblue"], lw=1)
plt.show()

<IPython.core.display.Javascript object>

## Başka Bir Veri Seti
Tek değişkenli bir veri setine geri dönelim. Sadece iki parametreyle, $w$, $b$, gradyan inişinin ne yaptığını daha iyi anlamak için maliyet fonksiyonunu bir kontur grafiği kullanarak çizebiliriz.


In [13]:
x_train = np.array([0., 1, 2, 3, 4, 5])
y_train = np.array([0,  0, 0, 1, 1, 1])

Öncekiler gibi, bu veriyi çizmek için bir yardımcı fonksiyon kullanacağız. $y=1$ etiketine sahip veri noktaları kırmızı çarpılarla, $y=0$ etiketine sahip veri noktaları ise mavi dairelerle gösterilecektir.


In [14]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
plt_tumor_data(x_train, y_train, ax)
plt.show()

<IPython.core.display.Javascript object>

Aşağıdaki grafikte şunları deneyin:
- $w$ ve $b$'yi üst sağdaki kontur grafiği üzerinde tıklayarak değiştirin.
    - Değişiklikler birkaç saniye sürebilir.
    - Sol üstteki grafikte maliyetin değişen değerini gözlemleyin.
    - Maliyet, her örnekteki kayıptan (dikey noktalı çizgiler) birikir.
- Gradyan inişi çalıştırmak için turuncu butona tıklayın.
    - Maliyetin düzenli bir şekilde azaldığını gözlemleyin (kontur ve maliyet grafikleri log(maliyet) cinsindendir).
    - Kontur grafiğine tıklamak, yeni bir çalışma için modeli sıfırlayacaktır.
- Grafiği sıfırlamak için hücreyi tekrar çalıştırın.


In [15]:
w_range = np.array([-1, 7])
b_range = np.array([1, -14])
quad = plt_quad_logistic( x_train, y_train, w_range, b_range )

<IPython.core.display.Javascript object>